## 🔗 Open This Notebook in Google Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DavidLangworthy/ds4s/blob/master/days/day04/notebook/day04_starter.ipynb)

# 🌍 Day 4 – Mapping Forest Change
Today’s learn–do loop adds spatial thinking: we will inspect the forest-cover data, build a static map as a checkpoint, then animate the story across three decades.

## 🗂️ Data Card: Global Forest Cover
- **Source:** World Bank (World Development Indicators), compiled from FAO Global Forest Resources Assessment.
- **Temporal coverage:** 1990–2020, annual.
- **Units:** Percentage of land area covered by forest per country (%).
- **Last updated:** 2024 data release (downloaded Nov 2024).
- **Method notes:** ISO-3 country codes with long-format year column; interpolated values for some small states.
- **Caveats:** Aggregated regions exist (e.g., “Africa Eastern and Southern”) — filter to ISO codes when mapping. Percentages do not reveal forest quality or biodiversity richness.
- **Integrity prompt:** Colour scales can over-emphasise small changes; commit to a perceptually uniform palette and keep the range fixed at 0–100%.

## Story Scaffold Reminder
- **Claim:** Which regional pattern or trend in forest change matters most?
- **Evidence:** Which years, countries, or deltas support that claim?
- **Visual:** How do palette, animation speed, and titles keep interpretation honest?
- **Takeaway:** Draft the conservation message you want viewers to repeat.

## Step 0 · Imports

In [ ]:
import pandas as pd
import plotly.express as px

from days.utils import (
    check_story_metadata,
    load_data,
    plots_directory,
    quick_diagnostics,
)

## Step 1 · Load and inspect the dataset

In [ ]:
forest = load_data("data/forest_area_long.csv")
quick_diagnostics(
    forest,
    expected_columns=["Country Name", "Country Code", "Year", "ForestPercent"],
    rows_between=(6000, 7000),
)
print("Expected: 31 years × ~215 countries ≈ 6650 rows.")

## Step 2 · Filter to ISO-coded countries and ensure numeric years

In [ ]:
country_rows = forest["Country Code"].str.len() == 3
forest_countries = forest.loc[country_rows].copy()
forest_countries["Year"] = forest_countries["Year"].astype(int)
quick_diagnostics(
    forest_countries.head(10),
    expected_columns=["Country Name", "Country Code", "Year", "ForestPercent"],
    rows_between=(6000, 6500),
    head_rows=5,
)
print("Check: ISO-3 codes only; Year column is int.")

## Step 3 · Interim static map
Before animating, confirm the geography renders correctly with a single year snapshot.

![Interim preview – world map shaded by forest cover.](../../plots/day04_solution_plot.png)

In [ ]:
latest_year = forest_countries["Year"].max()
fig_static = px.choropleth(
    forest_countries.query("Year == @latest_year"),
    locations="Country Code",
    color="ForestPercent",
    hover_name="Country Name",
    color_continuous_scale="YlGn",
    range_color=[0, 100],
    labels={"ForestPercent": "Forest cover (%)"},
    title=f"Draft: Global forest cover in {latest_year}",
)
fig_static.show(renderer="notebook")

## Step 4 · Story metadata

In [ ]:
TITLE = "Forest cover is declining in key biodiversity hotspots"
SUBTITLE = "Share of land area covered by forest, 1990–2020"
ANNOTATION = "Amazon and Southeast Asian countries lose >5 percentage points of forest in three decades."
SOURCE = "Source: FAO Global Forest Resources Assessment via World Bank WDI"
UNITS = "Units: Forest area as % of national land area"

check_story_metadata(
    TITLE=TITLE,
    SUBTITLE=SUBTITLE,
    ANNOTATION=ANNOTATION,
    SOURCE=SOURCE,
    UNITS=UNITS,
)

## Step 5 · Animated choropleth story
Animate across years, keep colour scales consistent, and add annotations that draw attention to rapid loss regions.

In [ ]:
fig = px.choropleth(
    forest_countries,
    locations="Country Code",
    color="ForestPercent",
    hover_name="Country Name",
    animation_frame="Year",
    color_continuous_scale="YlGn",
    range_color=[0, 100],
    labels={"ForestPercent": "Forest cover (%)"},
    title=f"{TITLE}<br><sup>{SUBTITLE}</sup>",
    template="plotly_white",
)
fig.update_layout(
    margin=dict(l=0, r=0, t=80, b=20),
    coloraxis_colorbar=dict(title="Forest %"),
    updatemenus=[
        dict(
            type="buttons",
            showactive=False,
            buttons=[dict(label="Play", method="animate", args=[None])],
        )
    ],
    sliders=[
        dict(
            currentvalue=dict(prefix="Year: "),
            transition=dict(duration=500),
        )
    ],
    annotations=[
        dict(
            text=ANNOTATION,
            x=0.01,
            y=0.02,
            xref="paper",
            yref="paper",
            showarrow=False,
            align="left",
            bgcolor="rgba(255,255,255,0.85)",
            bordercolor="#2b8a3e",
            borderwidth=1,
            font=dict(size=12),
        ),
        dict(
            text=f"{SOURCE} · {UNITS}",
            x=0.5,
            y=-0.08,
            xref="paper",
            yref="paper",
            showarrow=False,
            font=dict(size=11, color="#444444"),
        ),
    ],
)
fig.show(renderer="notebook")

## Step 6 · Interpret with the scaffold
- **Claim:** Tropical regions with mega-biodiversity continue to lose forest cover.
- **Evidence:** Brazil, Indonesia, and the Congo Basin visibly darken over the animation; only a handful of countries (e.g., China, parts of Europe) gain cover.
- **Visual:** A fixed colour scale and annotated callout ensure viewers compare like with like across time.
- **Takeaway:** “Protecting biodiversity means slowing forest loss in the Amazon and Southeast Asia — the map shows the hotspots still shrinking.”

### Limitations to note
- Percent cover does not measure forest quality; plantation expansion can mask biodiversity loss.
- Country-level aggregation hides subnational differences; consider pairing with region-level or deforestation-rate data.
- Animation speed may overwhelm some viewers; offer a static before–after comparison for accessibility.

## Step 7 · Export the interactive map

In [ ]:
export_path = plots_directory() / "day04_solution_plot.html"
fig.write_html(str(export_path))
print(f"💾 Saved interactive figure to {export_path}")